In [136]:
import pandas as pd
import numpy as np
import json
from matplotlib.path import Path
import datetime

In [361]:
dt = pd.read_csv('Data_3_provs.csv',sep=';')

In [362]:
dt = dt[dt.Provider != 0]

In [363]:
dt = dt.reset_index()

In [27]:
with open('carsharing_map.json', 'r') as outfile:
    carsharing_map = json.load(outfile)

In [46]:
for i in range(len(carsharing_map['features'])):
    names.append(carsharing_map['features'][i]['properties']['NAME'])

In [468]:
# рыба для конечного результата
df = pd.DataFrame({'MO_Name':names,'Weekday':np.nan,'Hour':np.nan})

weekdays = ['пн','вт','ср','чт','пт','сб','вс']

dt_full = pd.DataFrame(columns={'MO_Name','Weekday','Hour'})

for weekday in weekdays:
    for hour in range(24):
        df['Weekday'] = weekday
        df['Hour'] = hour
        dt_full = pd.concat([dt_full, df])

dt_full['Number'] = np.random.uniform(low=0,high=100,size=dt_full.shape[0])
dt_full = dt_full[['MO_Name','Weekday','Hour','Number']]
dt_full.to_csv('sample_data_result.csv',index=False,sep=';')

# данные для анализа

In [501]:
dt.head()

,index,Provider,Group,Longitude,StartTime,EndTime,Latitude,MinEpooch,MaxEpoch,MO_name
0,168,1,2553,37.610218,2016-02-18 00:28:21.737,2016-02-18 05:59:41.237,55.653446,30514,30624,Нагорный
1,169,1,2554,37.610138,2016-02-18 06:11:42.990,2016-02-18 13:38:12.500,55.653488,30628,30776,Нагорный
2,170,1,2555,37.785305,2016-02-18 16:03:00.683,2016-02-18 16:45:16.857,55.748455,30824,30838,Перово
3,171,1,2556,37.776054,2016-02-18 17:27:31.890,2016-02-19 03:27:23.177,55.746761,30852,31051,Перово
4,172,1,2557,37.772121,2016-02-19 03:45:29.040,2016-02-19 14:55:31.493,55.757855,31057,31279,Перово


In [ ]:
dt.StartTime = pd.to_datetime(dt.StartTime, format='%Y-%m-%d %H:%M:%S.%f')
dt.EndTime = pd.to_datetime(dt.EndTime, format='%Y-%m-%d %H:%M:%S.%f')

In [365]:
dt = dt[dt.StartTime > pd.to_datetime('2016-02-18',format ='%Y-%m-%d')]

In [ ]:
def reproject(latitude, longitude):
    """Returns the x & y coordinates in meters using a sinusoidal projection"""
    from math import pi, cos, radians
    earth_radius = 6371009 # in meters
    lat_dist = pi * earth_radius / 180.0

    y = [lat * lat_dist for lat in latitude]
    x = [long * lat_dist * cos(radians(lat)) 
                for lat, long in zip(latitude, longitude)]
    return x, y

def area_of_polygon(x, y):
    # возвращаем площадь полигода в квадратных километрах
    area = 0.0
    for i in range(-1, len(x)-1):
        area += x[i] * (y[i+1] - y[i-1])
    return abs(area) / 2.0 / 10 ** 6

Sq = []
for mo in range(len(carsharing_map['features'])):
    
    coords = pd.DataFrame(carsharing_map['features'][mo]['geometry']['coordinates'][0],columns = {'long','lat'})

    x,y=reproject(coords['lat'], coords['long'])
    S = area_of_polygon(x,y)
    Sq.append((carsharing_map['features'][mo]['properties']['NAME'],S))
    
Sq = pd.DataFrame(Sq,columns = {'MO_name','km_sq'})

In [ ]:
dt['HourStart'] = dt.StartTime.dt.hour
dt['HourEnd'] = dt.EndTime.dt.hour

dt['WeekdayStart'] = dt.StartTime.dt.weekday + 1
dt['WeekdayEnd'] = dt.EndTime.dt.weekday + 1

In [502]:
dt['timediff'] = dt['EndTime']-dt['StartTime']

In [262]:
dt['timediff'].max()

Timedelta('21 days 09:52:21.230000')

In [367]:
# восстановить район по координатам
def get_polygons(geojson):
    # функция возвращает многоугольники районов
    polygons = []
    for AO_num in range(len(geojson['features'])):
        AO_name = geojson['features'][AO_num]['properties']['NAME']
        AO_borders = geojson['features'][AO_num]['geometry']['coordinates']
        AO_num_polygons = len(AO_borders)
        # для каждого многоугольника в районе делаем лист ['название района','лист с координатами'],
        # добавляем в общий список
        for plg in range(AO_num_polygons):
            if AO_num_polygons == 1:
                plg_coordinates = [AO_name, AO_borders[plg]]
            else:
                plg_coordinates = [AO_name, AO_borders[plg][0]]
            polygons.append(plg_coordinates)
    return polygons

In [368]:
polygons = get_polygons(carsharing_map)
polygons = [[i[0],Path(i[1])] for i in polygons]

In [369]:
def get_dot_region(dot, polygons):
    # функция возвращает район для точки
    # dot: [longitude,latitude]
    regions = []
    for reg in polygons:
        if reg[1].contains_point([dot['Longitude'], dot['Latitude']]):
            regions.append(reg[0])
            break
    if regions == []:
        regions.append('Другой')
    return regions

In [370]:
# добавляем название района к точке 
# dt['AO_name'] = np.nan
st = datetime.datetime.now()
dt['MO_name'] = dt.ix[:,['Longitude','Latitude']].apply(lambda x: get_dot_region(x, polygons), axis=1)
en = datetime.datetime.now() - st
print(en)

0:06:03.060693


In [371]:
dt.MO_name = dt.MO_name.apply(lambda x: str(x).replace('[','').replace(']','').replace("'",''))

In [ ]:
dt = dt.reset_index()

In [523]:
dt.head()

,index,Provider,Group,Longitude,StartTime,EndTime,Latitude,MinEpooch,MaxEpoch,MO_name,timediff
0,168,1,2553,37.610218,2016-02-18 00:28:21.737,2016-02-18 05:59:41.237,55.653446,30514,30624,Нагорный,05:31:19.500000
1,169,1,2554,37.610138,2016-02-18 06:11:42.990,2016-02-18 13:38:12.500,55.653488,30628,30776,Нагорный,07:26:29.510000
2,170,1,2555,37.785305,2016-02-18 16:03:00.683,2016-02-18 16:45:16.857,55.748455,30824,30838,Перово,00:42:16.174000
3,171,1,2556,37.776054,2016-02-18 17:27:31.890,2016-02-19 03:27:23.177,55.746761,30852,31051,Перово,09:59:51.287000
4,172,1,2557,37.772121,2016-02-19 03:45:29.040,2016-02-19 14:55:31.493,55.757855,31057,31279,Перово,11:10:02.453000


In [539]:
def get_time_intervals(start, end, reg, provider):
    st = start - datetime.timedelta(minutes=start.minute,
                           seconds=start.second,
                           microseconds=start.microsecond)
    en = end - datetime.timedelta(minutes=end.minute,
                             seconds=end.second,
                             microseconds=end.microsecond)
    step = datetime.timedelta(hours=1)
    
    result = []
    while st < en:
        result.append((reg,st.weekday()+1,st.hour, provider))
        st += step
    
    return result

# разбивка по провайдерам

In [533]:
c1 = dt.Provider == 1
c2 = dt.Provider == 2
c3 = dt.Provider == 3
c12 = (dt.Provider == 1) & (dt.Provider == 2)
c23 = (dt.Provider == 3) & (dt.Provider == 2)
c12 = (dt.Provider == 1) & (dt.Provider == 3)

In [536]:
dt.head()

,index,Provider,Group,Longitude,StartTime,EndTime,Latitude,MinEpooch,MaxEpoch,MO_name,timediff
0,168,1,2553,37.610218,2016-02-18 00:28:21.737,2016-02-18 05:59:41.237,55.653446,30514,30624,Нагорный,05:31:19.500000
1,169,1,2554,37.610138,2016-02-18 06:11:42.990,2016-02-18 13:38:12.500,55.653488,30628,30776,Нагорный,07:26:29.510000
2,170,1,2555,37.785305,2016-02-18 16:03:00.683,2016-02-18 16:45:16.857,55.748455,30824,30838,Перово,00:42:16.174000
3,171,1,2556,37.776054,2016-02-18 17:27:31.890,2016-02-19 03:27:23.177,55.746761,30852,31051,Перово,09:59:51.287000
4,172,1,2557,37.772121,2016-02-19 03:45:29.040,2016-02-19 14:55:31.493,55.757855,31057,31279,Перово,11:10:02.453000


In [540]:
%%time
# собираем общую таблицу
r = []
for row in range(dt.shape[0]):
    r.append(get_time_intervals(dt['StartTime'][row], dt['EndTime'][row], dt['MO_Name'][row],dt['Provider'][row]))

Wall time: 19.6 s


In [542]:
%%time
list_10k = []
n = 0
while n < len(r):
    result_dt = pd.DataFrame(columns={'MO_Name','Weekday','Hour','Provider'})
    for d in r[n:n+1000]:
        
        result_dt = pd.concat([result_dt, pd.DataFrame(r[n], columns={'MO_name','Weekday','Hour','Provider'})])
        n += 1
    #     for row in d:
    #         result_dt = result_dt.append(pd.DataFrame({'MO_name':[row[0]],'Weekday':[row[1]],'Hour':[row[2]]}))
    list_10k.append(result_dt)
    if n %10000==0:
        print(n)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
Wall time: 5min 11s


In [563]:
result_dt = pd.DataFrame(columns={'MO_Name','Weekday','Hour','Provider'})
for frame in list_10k:
    result_dt = result_dt.append(frame)

result_dt.columns = ['MO_Name','Weekday','Hour','Provider']
result_dt_gr = result_dt.groupby(['MO_Name','Weekday','Hour','Provider']).size().reset_index()
result_dt_gr.columns = ['MO_Name','Weekday','Hour','Provider','Count']

In [564]:
minDate = dt.StartTime.min()
maxDate = dt.StartTime.max()

t1 = pd.DataFrame(get_time_intervals(minDate, maxDate, 'reg',1))
t2 = pd.DataFrame(get_time_intervals(minDate, maxDate, 'reg',2))
t3 = pd.DataFrame(get_time_intervals(minDate, maxDate, 'reg',3))
t1.columns=['c','Weekday','Hour','Provider']
t2.columns=['c','Weekday','Hour','Provider']
t3.columns=['c','Weekday','Hour','Provider']

t = pd.concat([t1,t2,t3])
del t['c']

In [565]:
t=t.groupby(['Weekday','Hour','Provider']).size().reset_index()
t.columns = ['Weekday','Hour','Provider','N']

In [590]:
result_dt_gr.head()

,MO_Name,Weekday,Hour,Number,Provider
0,Академический,1,0,0.234679,1
1,Академический,1,0,0.093871,2
2,Академический,1,0,0.109517,3
3,Академический,1,1,0.234679,1
4,Академический,1,1,0.125162,2


In [567]:
result_dt_gr = pd.merge(result_dt_gr, t, on = ['Weekday','Hour','Provider'], how= 'left')

In [ ]:
result_dt_gr = pd.merge(result_dt_gr,Sq, on ='MO_name', how='left')

result_dt_gr['Number'] = result_dt_gr['Count'] / result_dt_gr['N'] / result_dt_gr['km_sq']

result_dt_gr = result_dt_gr[['MO_Name','Weekday','Hour','Number','Provider']]
# result_dt_gr.columns = ['MO_Name','Weekday','Hour','Number','Provider']

In [602]:
result_dt_gr.Weekday = result_dt_gr.Weekday.map({1:'пн',2:'вт',3:'ср',4:'чт',5:'пт',6:'сб',7:'вс'})

In [ ]:
dt_full1 = dt_full.copy()
dt_full1['Provider'] = 1

dt_full2 = dt_full.copy()
dt_full2['Provider'] = 2

dt_full3 = dt_full.copy()
dt_full3['Provider'] = 3

dt_full_provider = pd.concat([dt_full1,dt_full2,dt_full3])
del dt_full_provider['Number']

In [604]:
result_dt_gr.head()

,MO_Name,Weekday,Hour,Number,Provider
0,Академический,пн,0,0.234679,1
1,Академический,пн,0,0.093871,2
2,Академический,пн,0,0.109517,3
3,Академический,пн,1,0.234679,1
4,Академический,пн,1,0.125162,2


In [603]:
dt_full_actual = pd.merge(dt_full_provider, 
                          result_dt_gr, 
                          how = 'left', 
                          on = ['MO_Name','Weekday','Hour','Provider'])

# dt_full_actual
dt_full_actual.fillna(0,inplace=True)

In [605]:
dt_full_actual.to_csv('real_data_result_with_provider.csv',index=False,sep=';')

In [469]:
dt_full_actual.Number.max()

5.505045488819615

In [471]:
# создаем пустую карту Москвы
import folium

In [520]:
MSC_COORDINATES = (55.75, 37.67) 

Mow_map = folium.Map(location=MSC_COORDINATES, zoom_start=10)

Mow_map.choropleth(geo_str=json.dumps(carsharing_map),
                    data = dtt,
                    columns = ['MO_Name', 'Number'],
                    key_on = 'feature.properties.NAME',
                    fill_color = 'RdYlGn',
                   threshold_scale =  list(dtt.Number.quantile([0,0.25,0.5,0.75,1])),#[0, 1, 2, 3, 4.15],
                    fill_opacity = 0.75, # Прозрачность
                    line_opacity = 0.5, # толщина линии
                    legend_name = 'Количество машин по округам'
                  )
    
#Mow_map.choropleth(geo_path=mos_reg_data_MO)
Mow_map.save(outfile='carsharing_map.html')

In [ ]:
dtt = dt_full_actual[(dt_full_actual.Weekday == 'вт') & (dt_full_actual.Hour == 11)][['MO_Name','Number']]
#dtt.ix[dtt.Number > 3,'Number'] = 3

In [519]:
list(dtt.Number.quantile([0,0.25,0.5,0.75,1]))

[0.0,
 0.3440579380125266,
 0.54662723169580363,
 0.76576204798996605,
 4.1496365616481183]

In [482]:
dtt.head()

,MO_Name,Number
928,Филёвский Парк,0.434490
929,Щукино,0.279059
930,Нагатинский Затон,0.195782
931,Дмитровский,0.200786
932,Тёплый Стан,0.329866


In [521]:
dt.head()

,index,Provider,Group,Longitude,StartTime,EndTime,Latitude,MinEpooch,MaxEpoch,MO_name,timediff
0,168,1,2553,37.610218,2016-02-18 00:28:21.737,2016-02-18 05:59:41.237,55.653446,30514,30624,Нагорный,05:31:19.500000
1,169,1,2554,37.610138,2016-02-18 06:11:42.990,2016-02-18 13:38:12.500,55.653488,30628,30776,Нагорный,07:26:29.510000
2,170,1,2555,37.785305,2016-02-18 16:03:00.683,2016-02-18 16:45:16.857,55.748455,30824,30838,Перово,00:42:16.174000
3,171,1,2556,37.776054,2016-02-18 17:27:31.890,2016-02-19 03:27:23.177,55.746761,30852,31051,Перово,09:59:51.287000
4,172,1,2557,37.772121,2016-02-19 03:45:29.040,2016-02-19 14:55:31.493,55.757855,31057,31279,Перово,11:10:02.453000


In [522]:
dt_full_actual

,MO_Name,Weekday,Hour,Number
0,Филёвский Парк,пн,0,0.517691
1,Щукино,пн,0,0.390682
2,Нагатинский Затон,пн,0,0.289011
3,Дмитровский,пн,0,0.276080
4,Тёплый Стан,пн,0,0.574606
5,Ховрино,пн,0,1.111829
6,Ломоносовский,пн,0,0.883905
7,Можайский,пн,0,0.551371
8,Строгино,пн,0,0.211781
9,Крылатское,пн,0,0.555621
